# 1.1 Introduction
The secure element manifest format is designed to convey the unique information about a secure elements including unique ID (e.g. serial number), public keys, and certificates. The signed manifest file
generated can be used to register the device to cloud providers.

## 1.2.Tutorial Prerequisites:
    

The following code is runs on python 3.x environment. The folowing modules are also needed to run the example

1. cryptoauthlib
2. cryptography

The next step of the tutorial with attempt to load all the necessary modules and their dependencies on your machine. If the above modules are already installed you can safely step over the next Tutorial step.

> <font color = green>pip install -U module_name</font>

In [ ]:
!python -m pip install -r ../../requirements.txt

## 1.3. Load the necessary modules and helper functions

This Tutorial uses a number of python helper functions that are loaded by the following cell.
The below code does the following things:
1. Setup of cryptography module
2. Adds helper functions to get device name and device type

In [ ]:
from cryptoauthlib import *
from common import *
from manifest_generation_helper import *
from IPython.display import HTML

# 2. Tutorial

This tutorial will guide you through the following steps:
1. Establish the communication between the host (PC) and the ECC608A
2. Generate the Manifest file


## 2.1. Establish the communication between the host (PC) and the ECC608A
1. Initialize the library which also loads it
2. Check if the correct device is connected to the PC. If the wrong device is connected, a ValueError excpetion will be raised.

### 2.1.1. Select the CryptoAuthLib Hardware Abstraction layer to KIT USB HID

In [ ]:
cfg = cfg_ateccx08a_kithid_default()
cfg.devtype = get_device_type_id('ATECC608A')
cfg.cfg.atcahid.dev_interface = ATCAKitType.ATCA_KIT_I2C_IFACE
cfg.cfg.atcahid.dev_identity = 0x6A #TNGTLS device I2C address
print("Set devtype as ECC608A - Successfull")

### 2.1.2. Establish the connection with the ECC608A-TNGTLS device on the USB dongle and check its type
<font color=red>If the next Tutorial step yields an exception, the ECC608A is not correctly connected or of the wrong type.</font>

In [ ]:
# Initialize interface
assert atcab_init(cfg) == ATCA_SUCCESS, "Can't connect to the USB dongle"

# Get connected device type
info = bytearray(4)
assert atcab_info(info) == ATCA_SUCCESS, "Can't read the ECC608A device information"
dev_type = get_device_type_id(get_device_name(info))

# Checking if the connected device matches with selected device
if dev_type != cfg.devtype:
    assert atcab_release() == ATCA_SUCCESS
    raise ValueError('Device not supported')
    
print("Device initialization - Successful")

## 2.2. Generate the Manifest file


### 2.2.1. Create a signing key and certificate that will be used to sign entries in a manifest
Manifests downloaded from Microchip Direct will be signed by Microchip

In [ ]:
logger_cert = create_log_signer()

### 2.2.2. Create the manifest file from the extracted device information
This step can take a little bit of time as all the certificates are extracted.

<font color=red>If the next Tutorial step yields an exception, the ECC608A is not correctly connected or of the wrong type.</font>

In [ ]:
manifest_data, manifest_name = tng_data()

### 2.2.3 Download Artifacts

In [ ]:
def create_download_link(file_data, file_name, title):  
    b64 = base64.b64encode(file_data)
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=file_name)
    return HTML(html)

#### 2.2.3.1 Download Manifest
Click the generated link to download the manifest

In [ ]:
create_download_link(manifest_data, manifest_name, 'Download Manifest')

#### 2.2.3.2 Download Verification Certificate
Click the generate link to download the manifest validation certificate

In [ ]:
create_download_link(logger_cert, 'logger.pem', 'Download Logger Certificate')